## News Scraping and Classification

#### We will be scraping the news from economic times for the month of August 2020

In [1]:
#importing the libaries. We will be using selenium webdriver for this

import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
#creating the input function for the data to be scrape

a=input('Enter the year you want to scrap the new for: ')
b=input('Enter the month in digit only ,i.e for Jan enter 1 or for Nov enter 11: ')
c=input('The number of days the month have: ')
url='https://economictimes.indiatimes.com/archive/year-{},month-{}.cms'.format(a,b)

Enter the year you want to scrap the new for: 2020
Enter the month in digit only ,i.e for Jan enter 1 or for Nov enter 11: 8
The number of days the month have: 31


In [3]:
print(url)

https://economictimes.indiatimes.com/archive/year-2020,month-8.cms


In [3]:
driver=webdriver.Chrome('chromedriver.exe')
driver.get(url)
driver.maximize_window()
time.sleep(2)

#1st we will scrape all the dates url
date_urls=[]
dates=[]

for i in driver.find_elements_by_xpath("//td[@align='center']/a"):
    date_urls.append(i.get_attribute('href'))
for i in date_urls:
    if i not in dates:
        dates.append(i)
if len(dates)!=c:
    dates=dates[1:]

print(len(dates))

31


In [ ]:
# creating the empty list for date to be store
news_date=[]
news_author=[]
news_topic=[]
news_headlines=[]
news_description=[]

In [ ]:
#creating the function

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "eager" #since economictimes has long loading time. This command will start scraping once 
driver = webdriver.Chrome(desired_capabilities=capa)  #the html is loaded.

new_dates=[]
for m in range(1,len(31)):
    driver.get(dates[m-1])
    
    for j in driver.find_elements_by_xpath('//ul[@class="content"]/li/a'):
        new_dates.append(j.get_attribute('href')) #appending all the news link of the particular day
    for k in new_dates:
        driver.get(k)
        
    
               
        date=driver.find_elements_by_xpath('//div[@class="dtc vam artByline"]/time')
        for i in date:
            if i.text is None:
                news_date.append("--")
            else:
                news_date.append(i.text.split(': ')[1])
        
#certain news only for paid subscribers and it will create the uneven list. The below command will ignore such page
        if len(news_date)!=len(news_topic)+1:
            news_date.append("--")
            news_author.append('--')
            news_topic.append("--")
            news_headlines.append("--")
            news_description.append("--")
            continue            
        
        author=driver.find_elements_by_xpath('//span[@class="ag"]')
        for i in author:
            news_author.append(i.text)
                
           
        topic=driver.find_elements_by_xpath('//div[@class="clr breadCrumb contentwrapper"]/span[3]')
        for i in topic:
            news_topic.append(i.text)
                
        
        headlines=driver.find_elements_by_xpath('//h1[@class="artTitle font_faus"]')
        for i in headlines:
            news_headlines.append(i.text)
        
        time.sleep(2) 
        try:
            try:news_description.append(driver.find_element_by_xpath("//div[@class='artText medium']").text.replace('\n','').replace('\'',''))
            except:news_description.append(driver.find_element_by_xpath("//div[@class='artText m']").text.replace('\n','').replace('\'',''))    
        except NoSuchElementException:
            news_description.append("--")
                        
        print((len(news_date),len(news_author),len(news_topic),len(news_headlines),len(news_description)))
    
    print('Scraping of date', m, 'completed')
    new_dates=[]

In [12]:
#lets create the data frame
df=pd.DataFrame({"Date and time of upload":news_date,"Author name":news_author,"Topic or Vertical":news_topic,
                 "Headlines":news_headlines,"Description":news_description})    

In [16]:
#removing the null values from the dataframe
index_names = df[ df['Description'] == "--" ].index
df.drop(index_names, inplace = True)

In [18]:
#creating the csv for scrape date
df.to_csv('News.csv')

In [19]:
#creating the JSON file
df.to_json('News.json', orient = 'split', compression = 'infer', index = 'true')

In [20]:
#creating the excel file
df.to_excel('NewsAug20.xlsx')

In [14]:
df=pd.read_csv('News.csv')

In [17]:
df

,Unnamed: 0,Date and time of upload,Author name,Topic or Vertical,Headlines,Description
0,0,"Aug 01, 2020, 12:03 AM IST",ET Bureau,Politics,Erratic rainfall hits crop planting as sowing ...,NEW DELHI: Erratic rainfall has slowed down cr...
1,1,"Aug 01, 2020, 12:03 AM IST",ET Bureau,Economy,Continuation of safeguard duties on imports fr...,NEW DELHI: The governments decision to continu...
2,2,"Aug 01, 2020, 12:47 AM IST",Bloomberg,International,Microsoft is said to be in talks to buy TikTok...,By Kurt WagnerMicrosoft Corp. is exploring an ...
3,3,"Aug 01, 2020, 07:22 AM IST",ET Bureau,Cons. Products,Import restrictions on colour television to gi...,New Delhi: TV manufacturers like Chinas Xiaomi...
4,4,"Aug 01, 2020, 07:45 AM IST",ET Bureau,Politics,Rajasthan political crisis: BJP hopes rest on ...,New Delhi: Rajasthan governor Kalraj Mishra ha...
...,...,...,...,...,...,...
9306,9306,"Sep 01, 2020, 05:31 PM IST",PTI,GST,Govt extends FY20 GST return filing date for c...,New Delhi:The government on Monday extended by...
9307,9307,"Aug 31, 2020, 10:48 PM IST",ET Bureau,Services,Office space leasing dropped by 60% in six cit...,NEW DELHI: Office space absorption in six majo...
9308,9308,"Aug 31, 2020, 10:55 PM IST",ET Bureau,Stocks,RBI may propose tweaks to Sarfaesi Act to brid...,Mumbai: The Reserve Bank of India may propose ...
9309,9309,"Aug 31, 2020, 10:53 PM IST",ET Bureau,Politics,"Some relief as Tamil Nadu reports under 6,000 ...",Chennai: Tamil Nadu saw under 6000 cases on Mo...


In [24]:
print((len(news_date),len(news_author),len(news_topic),len(news_headlines),len(news_description)))

(9290, 9290, 9290, 9290, 9290)


##### Completed